In [1]:
# imports
import huggingface as hf
import pandas as pd
import numpy as np
from datasets import load_dataset, concatenate_datasets, Dataset
import torch

In [2]:
# load hf dataset and convert to pandas dataframe
dataset = load_dataset("allenai/tulu-2.5-preference-data")
combined_dataset = concatenate_datasets([
    dataset["alpaca_farm_human_pref"],
    dataset["argilla_dpo_mix"],
    dataset["ultrafeedback_truthful_qa"]
])
df = pd.DataFrame(combined_dataset)
# display first few rows
df.head()

,chosen,rejected,source
0,"[{'role': 'user', 'content': 'Arrange the foll...","[{'role': 'user', 'content': 'Arrange the foll...",alpaca_farm_alpaca_human_preference
1,"[{'role': 'user', 'content': 'Explain how digi...","[{'role': 'user', 'content': 'Explain how digi...",alpaca_farm_alpaca_human_preference
2,"[{'role': 'user', 'content': 'Task: Given an a...","[{'role': 'user', 'content': 'Task: Given an a...",alpaca_farm_alpaca_human_preference
3,"[{'role': 'user', 'content': 'Describe a chara...","[{'role': 'user', 'content': 'Describe a chara...",alpaca_farm_alpaca_human_preference
4,"[{'role': 'user', 'content': 'Rewrite this sen...","[{'role': 'user', 'content': 'Rewrite this sen...",alpaca_farm_alpaca_human_preference


In [3]:
# modify to have input, chosen and rejected as keys
# Function to extract input, chosen, rejected text
def extract_text(example):
    # Extract user content (input)
    chosen_user = next((m["content"] for m in example["chosen"] if m["role"] == "user"), "")
    rejected_user = next((m["content"] for m in example["rejected"] if m["role"] == "user"), "")

    # Usually user messages are identical in both — pick one
    user_input = chosen_user or rejected_user

    # Extract assistant content (responses)
    chosen_assistant = next((m["content"] for m in example["chosen"] if m["role"] == "assistant"), "")
    rejected_assistant = next((m["content"] for m in example["rejected"] if m["role"] == "assistant"), "")

    return {
        "input": user_input.strip(),
        "chosen": chosen_assistant.strip(),
        "rejected": rejected_assistant.strip(),
    }

# Apply to dataset
combined_dataset = combined_dataset.map(extract_text)

# Remove old columns if desired
combined_dataset = combined_dataset.remove_columns(["source"])

In [6]:
# convert to df
df = pd.DataFrame(combined_dataset)
# display first few rows
df.head()
# shuffle dataset
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
print(f"Total examples: {len(combined_dataset)}")

Total examples: 17247


# Prompt Classification

In [7]:
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



In [ ]:
from transformers import pipeline
from datasets import Dataset
from tqdm.auto import tqdm
import os

# Define labels and concise system prompt
labels = ["factual", "creative"]
system_prompt = """Classify the following query as either 'factual' or 'creative'. 'Factual' queries seek information about a particular fact. These include questions about events, definitions, explanations, factual queriesor data that can be verified or researched. 'Creative' queries request imaginative, creative content. These include prompts for storytelling, brainstorming, opinions, essay writing, etc.

Examples:
Query: What is the happiest country in the world?
Answer: factual

Query: Write a short story about a dragon and a knight.
Answer: creative

Query: """


# Initialize zero-shot classifier (use GPU if available)
classifier = pipeline(
    "zero-shot-classification",
    model="facebook/bart-large-mnli",
    device_map="auto",
    batch_size=32,          # tune based on GPU memory
    truncation=True
)

# # use llama-1b model for classification
# classifier = pipeline(
#     "text-classification",
#     model="meta-llama/Llama-3.2-1B",
#     device_map="auto",
#     batch_size=32,          # tune based on GPU memory
#     truncation=True, 
#     # token="hf_xxx"  # replace with your hf token if needed
# )

# Prepare batched function
def batch_classify(batch):
    inputs = [system_prompt + text + "\nAnswer: " for text in batch["input"]]
    # prompt a small LLM to classify
    results = classifier(inputs, candidate_labels=labels, multi_label=False)
    # store label, factual and creative scores
    batch["label"] = [res["labels"][0] for res in results]
    batch["factual_score"] = [
        res["scores"][res["labels"].index("factual")] for res in results
    ]
    batch["creative_score"] = [
        res["scores"][res["labels"].index("creative")] for res in results
    ]
    return batch

# convert pandas dataframe back to hf dataset
dataset = Dataset.from_pandas(df)
# # take first 2 samples
# dataset = dataset.select(range(60, 80))
# Parameters
save_dir = "../data/dpo_factual_creative_split"
save_every = 640  # Save every 640 samples

# Create save directory if it doesn’t exist
os.makedirs(save_dir, exist_ok=True)

# Process dataset in chunks and save periodically
chunk_size = save_every
total = len(dataset)

for start in tqdm(range(0, total, chunk_size)):
    end = min(start + chunk_size, total)
    chunk = dataset.select(range(start, end))
    chunk = chunk.map(batch_classify, batched=True, batch_size=32)
    
    # Save chunk to disk
    chunk.save_to_disk(f"{save_dir}/chunk_{start}_{end}")
    print(f"✅ Saved chunk {start}–{end} to disk")

Some parameters are on the meta device because they were offloaded to the disk.
Device set to use mps


  0%|          | 0/27 [00:00<?, ?it/s]

Parameter 'function'=<function batch_classify at 0x5448d5d00> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only shown once. Subsequent hashing failures won't be shown.


Map:   0%|          | 0/640 [00:00<?, ? examples/s]

In [ ]:
from datasets import concatenate_datasets
# merge all chunks back into a single dataset
all_chunks = []
for start in range(0, total, chunk_size):
    end = min(start + chunk_size, total)
    chunk = Dataset.load_from_disk(f"{save_dir}/chunk_{start}_{end}")
    all_chunks.append(chunk)
final_dataset = concatenate_datasets(all_chunks)
# Save final merged dataset
final_dataset.save_to_disk(f"{save_dir}/final_factual_creative_dataset")

print("🎉 All chunks processed and saved successfully!")

Saving the dataset (0/1 shards):   0%|          | 0/20 [00:00<?, ? examples/s]

🎉 All chunks processed and saved successfully!


In [ ]:
# load final merged dataset and display first few rows
final_dataset = Dataset.load_from_disk(f"{save_dir}/final_factual_creative_dataset")
final_df = final_dataset.to_pandas()
# display first 10 rows
# increase column display width
pd.set_option('display.max_colwidth', 6000)
final_df.head(10)

,chosen,rejected,input,label,factual_score,creative_score
0,"No, there is no global maximum on the interval [0, 1] for the function f(x) = 1/x on (0, 1] and f(x) = 3 if x = 0. A global maximum is a value of the function that is greater than or equal to all other values of the function on the domain. Since f(x) approaches positive infinity as x approaches zero from the right (and equals 3 when x equals zero), there is no value of x where f(x) is largest on the interval [0, 1]. Therefore, there is no global maximum on this interval for function f(x) = 1/x on (0, 1) and f(x) = 3 if x = 0.","To determine if there is a global maximum on the interval $[0, 1]$ for the function $f(x) = \begin{cases} 3, & \text{if } x = 0, \\ \frac{1}{x}, & \text{if } x \in (0, 1] \end{cases}$, we need to examine the behavior of the function on this closed interval and compare different values of $f(x)$.\n\nFirst, let's consider the behavior of $f(x)$ when $x \in (0, 1]$. As $x$ gets closer to $1$, $\frac{1}{x}$ becomes larger since the denominator becomes smaller. In this portion of the function's domain, the function decreases as $x$ approaches $1$ from the right, but there is no maximum value because the limit of the function as $x$ approaches $1$ is infinity.\n\nNext, let's examine $f(x)$ at $x = 0$. Here, the function takes the constant value of $f(0) = 3$. This constant value is predefined and not determined by the behavior of the function in the interval $(0, 1]$.\n\nThere is no given definition of a global maximum for which we need to compare different local or relative maxima on the interval $[0, 1]$. Since the function changes types at the endpoint $x = 0$, discerning whether there is a global maximum across the entire interval $[0, 1]$ becomes less straightforward. However, in this particular case, based on the definition and behavior of the function, we can conclude the following:\n\nThere is no global maximum for the function $f(x)$ on the interval $[0, 1]$ when we consider the entire function, including both definitions. The constant value of $f(0) = 3$ is not larger than any value of $f(x)$ for $x \in (0, 1]$, in which case $f(x)$ approaches infinity as $x$ approaches $1$ from the right. Furthermore, since $f(x)$ decreases on the interval $(0, 1]$, there are no local maxima or relative maxima for this portion of the function's domain.\n\nIn summary, there is no global maximum for the function $f(x) = \begin{cases} 3, & \text{if } x = 0, \\ \frac{1}{x}, & \text{if } x \in (0, 1] \end{cases}$ on the interval $[0, 1]$, considering the entire function, including its different definitions on the subintervals $[0, 0]$ and $(0, 1]$.\n\nHowever, given that we're specifying an interval, one could make the argument that we are only speaking of the function $f(x) = \frac{1}{x}$ in the interval $(0, 1]$. In that case, where we only focus on this portion of the function, there is still no local or relative maximum or global maximum in the interval $(0, 1]$.","Let $f(x) = 1/x$ on $(0, 1]$ and $f(x) = 3$ if $x = 0$. Is there a global maximum on interval $[0, 1]$?",creative,0.466562,0.533438
1,"Facts are verifiable statements that can be objectively proven, while opinions are subjective, personal viewpoints that cannot necessarily be substantiated.","Facts are verifiable statements of objective reality, whereas opinions are subjective views or judgments held by individuals. Facts are based on evidence, while opinions do not necessarily need to be backed up by any evidence.",Explain the difference between facts and opinions in two sentences.,factual,0.505290,0.494710
2,"Once upon a time, deep in the heart of the forest lived a mythical creature named Charlotte. Charlotte was a majestic beast, with feathers of bright yellow, a long, slender tail, and wings that spread across the night sky. \n\nCharlotte had built her home in a tall tree, and she was often seen flying through the darkened sky, lighting up the sky as if it were morning. It was said that Cha

# embeddings

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

def embed(texts):
    return model.encode(texts, normalize_embeddings=True).tolist()

dataset = dataset.map(
    lambda x: {
        "prompt_emb": embed([x["input"]])[0],
        "chosen_emb": embed([x["chosen"]])[0],
        "rejected_emb": embed([x["rejected"]])[0],
    }
)
dataset.save_to_disk("distilabel-intel-orca-dpo-pairs-embeddings")

## Composition - 

- truthfulQA - ~900 factual truth testing questions (with correct and incorrect)
- flan_v2_niv2 - ~15k prompts with correct answer
- evol_instruct - good mix, but no correct/incorrect. 
- sharegpt - ~20k real prompts and responses. no correct/incorrect.
- ultrachat - variety of qs. no correct/incorrect. 